# DDPLKO Moduł 7 - praca domowa - HPO

To siódma praca domowa w Programie szkoleniowym Klasyfikacja obrazu od Deep Drive PL

Twoim zadaniem w tym module będzie jendo z 2:

Opcja 1:
- [ ] Na bazie pracy domowej z Transfer learningiem, data augmentation
- [ ] Wykorzystaj HPO (np. Optunę, Hyperopta bądź KerasTuner) by dobrać parametry treningu (optymalizator, LR), data augmentation (zakresy wartości)
- [ ] Udostępnij wykres na Discordzie `#klasyfikacja-wyniki` (val acc - porównanie model przed i po HPO)

Opcja 2 (mniejsza moc obliczeniowa):
- [ ] Na bazie pracy domowej ze zbiorem **QuickDraw**
- [ ] Wykorzystaj HPO (np. Optunę, Hyperopta bądź KerasTuner) by dobrać parametry treningu (optymalizator, LR), i architektury sieci (liczba warstw, dropout, pooling)
- [ ] Pracuj na podzbiorze max 100k przykładów
- [ ] Udostępnij wykres na Discordzie `#klasyfikacja-wyniki` (val acc - porównanie model przed i po HPO)

Możesz extra (czyli opcjonalne rzeczy):
- Spróbować użyć AutoKeras by znaleźć architekturę dla problemu

In [1]:
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback, TensorBoardCallback


import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set(style='whitegrid',font_scale=1.5)

for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
import tensorflow_datasets as tfds

In [3]:
(ds_train, ds_val),info = tfds.load(
    "resisc45",
    split=["train[:70%]", "train[70%:100%]"],
    as_supervised=True,  # Include labels
    with_info=True
)

print('Train:',len(ds_train))
print('Val:',len(ds_val))

Train: 22050
Val: 9450


In [4]:
img_size = 256
batch_size=32

size = (img_size, img_size)

ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_val = ds_val.map(lambda image, label: (tf.image.resize(image, size), label))
ds_val = ds_val.batch(batch_size=batch_size, drop_remainder=True)

In [5]:
class_names = info.features['label'].names

num_classes = len(class_names)
num_classes

45

# Optuna

In [6]:
def getModel(lr, pooling, flatten, convs):
    l = [tf.keras.layers.Conv2D(8,(3,3), padding='same', activation='relu',input_shape=(img_size, img_size,3)),]
    for i in range(int(convs)):
        if pooling=='avg':
            l.append(tf.keras.layers.AvgPool2D())
        if pooling=='max':
            l.append(tf.keras.layers.MaxPool2D())
        l.append(tf.keras.layers.Conv2D(2**(i+4),(3,3), padding='same', activation='relu'))
    if flatten=='global':
        l.append(tf.keras.layers.GlobalAveragePooling2D())
    else:
        l.append(tf.keras.layers.Flatten())
    l.append(tf.keras.layers.Dense(num_classes, activation='softmax'))
    
    model = tf.keras.models.Sequential(l)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

params={'lr':0.001,
        'pooling':'avg',
        'flatten':'global',
        'convs':5}

model = getModel(**params)
model.summary()
del model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
average_pooling2d (AveragePo (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
average_pooling2d_1 (Average (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
average_pooling2d_2 (Average (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1

In [7]:
epochs = 20

In [8]:
def objective(trial):
    convs = trial.suggest_int("convs", 0, 3)
    pooling = trial.suggest_categorical("pooling", ["no","avg", "max"])
    flatten = trial.suggest_categorical("flatten", ["global", "flatten"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)

    model = getModel(lr, pooling, flatten, convs)
    
    history = model.fit(ds_train, # 10% of data
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0,
                        validation_freq=1,
                        validation_data=(ds_val),
                        callbacks=[TFKerasPruningCallback(trial, "val_accuracy")])
    
    return history.history['val_accuracy'][-1] # return last val acc

In [9]:
study = optuna.create_study(direction="maximize")  # ==> maximize accuracy
study.optimize(objective, n_trials=100, timeout=None,
               callbacks=[TensorBoardCallback('optuna-keras-logs2','val_accuracy')])

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-11-30 21:38:41,789] A new study created in memory with name: no-name-1f5e74dc-48e3-483b-bb76-08e0121e1f5e
<ipython-input-9-a46fcee01c42>:3: ExperimentalWarning: TensorBoardCallback is experimental (supported from v2.0.0). The interface can change in the future.
  callbacks=[TensorBoardCallback('optuna-keras-logs2','val_accuracy')])
[I 2021-11-30 22:08:07,478] Trial 0 finished with value: 0.019915254786610603 and parameters: {'convs': 0, 'pooling': 'no', 'flatten': 'global', 'lr': 0.05927084942607407}. Best is trial 0 with value: 0.019915254786610603.
[I 2021-11-30 22:52:45,352] Trial 1 finished with value: 0.03241525590419769 and parameters: {'convs': 0, 'pooling': 'no', 'flatten': 'flatten', 'lr': 0.012095518300665343}. Best is trial 1 with value: 0.03241525590419769.
[I 2021-11-30 23:28:21,848] Trial 2 finished with value: 0.2814618647098541 and parameters: {'convs': 0, 'pooling': 'avg', 'flatten': 'global', 'lr': 0.0011369511690982135}. Best is trial 2 with value: 0.28146186

[I 2021-12-02 05:05:42,481] Trial 52 pruned. Trial was pruned at epoch 7.
[I 2021-12-02 05:09:01,319] Trial 53 pruned. Trial was pruned at epoch 0.
[I 2021-12-02 06:15:43,616] Trial 54 finished with value: 0.6260592937469482 and parameters: {'convs': 3, 'pooling': 'avg', 'flatten': 'global', 'lr': 0.0029347507592634267}. Best is trial 11 with value: 0.6713982820510864.
[I 2021-12-02 06:22:21,832] Trial 55 pruned. Trial was pruned at epoch 1.
[I 2021-12-02 06:25:40,458] Trial 56 pruned. Trial was pruned at epoch 0.
[I 2021-12-02 06:28:49,225] Trial 57 pruned. Trial was pruned at epoch 0.
[I 2021-12-02 06:32:04,614] Trial 58 pruned. Trial was pruned at epoch 0.
[I 2021-12-02 06:35:20,365] Trial 59 pruned. Trial was pruned at epoch 0.
[I 2021-12-02 06:38:23,996] Trial 60 pruned. Trial was pruned at epoch 0.
[I 2021-12-02 07:43:31,347] Trial 61 finished with value: 0.6426906585693359 and parameters: {'convs': 3, 'pooling': 'avg', 'flatten': 'global', 'lr': 0.0016557650763173051}. Best is t

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  67
  Number of complete trials:  33
Best trial:
  Value:  0.6713982820510864
  Params: 
    convs: 3
    pooling: avg
    flatten: global
    lr: 0.0013234805544161227


In [10]:
from torch.utils import tensorboard

In [11]:
%reload_ext tensorboard
%tensorboard --logdir optuna-keras-logs2 --port 6015

# Summary / extra